# Berman Standard State Code Generator 

A notebook to generate spud files for Berman standard state models

Required system packages and initialization

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import time
import warnings
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../../')
SPUD_DIR = HOME_DIR+'/endmembers/'
try:
    os.makedirs(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
print(HOME_DIR,SPUD_DIR)

In [ ]:
reference = 'fo_sio2/notebooks/Generate_berman_endmembers.ipynb'

## Clean the Berman Data base for different models

* Here we'll just use  standard state models without lambda transitions or order-disorder terms


In [ ]:
df = pd.read_json('data/berman_1988.json')
df.columns

In [ ]:
# pull out column names required for just standard state model
cols_ss = ['Phase', 'Formula', 'sAbbrev', 'lAbbrev', 'H_TrPr', 'S_TrPr', 'V_TrPr',
           'k0', 'k1', 'k2', 'k3', 'v1', 'v2', 'v3', 'v4']
df = df[cols_ss]

Rename some columns and just extract rows for phases Forsterite and Fayalite

In [ ]:
phases = ['Fo', 'En', 'bQz', 'bCr']
df = df.rename(index=str,columns={'Phase':'name', 'Formula':'formula'})
df = df[df.sAbbrev.isin(phases)]
df

## Generate Sympy symbols and Coder parameters from the column headers

To construct our Gibbs free energy we need to create Sympy symbols and units for the parameters
* H_TrPr: Enthalpy at reference $T$ and $P$ in $J$
* S_TrPr: Entropy at reference T and P in $J/K$
* V_TrPr: Volume at reference T and P in $J/bar$
* k0-k3:  Coefficients in Berman heat capacity model with units [ 'J/K-m', 'J-K^(1/2)-m', 'J-K/m', 'J-K^2' ]
* v1-v4:  Coefficients in Berman Equation of state (V) with units [ '1/bar', '1/bar^2' , '1/K', '1/K^2']


In [ ]:
param_strings = df.columns[4:]
units = [ 'J', 'J/K', 'J/bar']+ [ 'J/K-m', 'J-K^(1/2)-m', 'J-K/m', 'J-K^2' ]+[ '1/bar', '1/bar^2' , '1/K', '1/K^2']
print('parameter\tunit')
for t in list(zip(param_strings, units)):
    print('{}\t\t {}'.format(t[0],t[1]))

We now construct coder parameters and sympy symbols from the parameter names and pass them to the local dictionary for use in other sympy expressions

In [ ]:
params = coder.set_coder_params(param_strings, units)
symbol_dict = coder.get_symbol_dict_from_params(params)

# load local dictionary
locals().update(symbol_dict)


## Create  model class

In [ ]:
model = coder.StdStateModel.from_type()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

## Standard State Potentials

### Define model Potentials for the Standard State Potentials
An expression for the Gibbs free energy, $G(T,P)$ or the Helmholtz energy $A(T,V)$ is constructed.  The expression may have multiple parts.  Often the heat capacity function is postulated, then integrated to yield expressions for the entahlpy, entropy, and in combination the energy potential. Then, an equation of state (EOS) is adopted and that term is integrated in pressure or volume and added to the heat capacity integrals. This proceedure is follwed here.
#### (1) $C_P$ integrals
The isobaric heat capacity terms parameterized as: $C_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3 $,
and in addition the reference condition third law entropy, $ S_{Tr,Pr} $, and enthalpy of formation from the
elements, $ \Delta H_{Tr,Pr} $, constitute additional parameters:

In [ ]:
# Heat Capacity
CpPr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3

Define the heat capacity contribution to the Gibbs free energy ...

In [ ]:
GPr = H_TrPr + sym.integrate(CpPr,(T,Tr,T)) - T*(S_TrPr + sym.integrate(CpPr/T,(T,Tr,T)))

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space

Sympy symbols and parameters

In [ ]:
GPrToP = sym.integrate(V_TrPr*(1+v1*(P-Pr)+v2*(P-Pr)**2+v3*(T-Tr)+v4*(T-Tr)**2),(P,Pr,P))

#### Define standard state G and parameters

In [ ]:
G = GPr + GPrToP
G

### Generate a coder model and write to spud files

In [ ]:
model.add_potential_to_model('G',G, params)

Exam settable values for the model dictionary

In [ ]:
model.get_values()                       

### Write out spud files for Berman endmembers for Berman endmembers

In [ ]:
for i, row in df.iterrows():
    values_dict = row.to_dict()
    # clean up names to make them coder compliant
    values_dict['name'] ='{}_berman'.format(values_dict['name'].title())
    values_dict['name'] = values_dict['name'].replace('-','_')
    print(values_dict['name'])
    values_dict['reference']=reference
    model.set_values(values_dict)
    model.to_xml(path=SPUD_DIR)
